In [ ]:
import folium
import pandas as pd

#create folium map m
m = folium.Map(tiles = 'cartodbdark_matter')
folium.TileLayer("", name="None", attr="blank").add_to(m)
m.add_child(folium.map.LayerControl())


def add_to_map(response, opacities=0.1):
    '''
    uses coordinates to plot lines one by one onto a folium map

    Params
    response- API JSON request response containing all coordinates to form the line
    opacities- a list of the opacity to plot the line on the map

    Return
    m- folium map with lines plotted
    '''
    assert isinstance(response, dict)
    assert isinstance(opacities, float)

    coordinates = response['features'][0]['geometry']['coordinates']
    points = [(i[1], i[0]) for i in coordinates[0]]
    folium.PolyLine(points, color = 'red',  weight=1, opacity=opacities).add_to(m)

   # create optimal zoom
    df = pd.DataFrame(coordinates[0]).rename(columns={0:'Lon', 1:'Lat'})[['Lat', 'Lon']]
    sw = df[['Lat', 'Lon']].min().values.tolist()
    ne = df[['Lat', 'Lon']].max().values.tolist()
    m.fit_bounds([sw, ne])

    return m


def create_map(csv_file):
    '''
    plots a csv file of coordinates as lines onto folium map m

    params
    csv_file- str location of csv file of coordinates
    '''
    import ast
    data = pd.read_csv(csv_file)

    opacities = data['count'].values
    opacities = (opacities - opacities.min()) / (opacities.max() - opacities.min())
    opacities = opacities * 0.5 + 0.5

    for i in range(len(data)):
        if data['coordinates'].iloc[i] != "error":
            data_i = data['coordinates'].iloc[i]
            json_data = ast.literal_eval(data_i)
            
            add_to_map(json_data, opacities[i])

create_map('../processed_datasets/sf_final.csv')

#calling m displays the map
m